In [2]:
import numpy as np

In [36]:
# Specify dimensions

print("Please provide your desired grid dimension (dim X dim)\n")

s = int(input())

print("\n")

# Create basic structure

for x in range(s+1):
    grid_world = ["O"] * (x * x)

# Create obstacles

grid_world = np.array(grid_world)

num_obs = int((x*x)/8) # specify number of obstacles

indices = np.random.choice(np.arange(grid_world.size), replace=False,
                           size=num_obs)

grid_world[indices] = "X"

grid_world = grid_world.reshape(((x, x)))

# Show

print("Gridworld:\n")
print(grid_world)
print("\n")

Please provide your desired grid dimension (dim X dim)

15


Gridworld:

[['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'X' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'X' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'X' 'O' 'O' 'O' 'X' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'X' 'O' 'O']
 ['O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'X' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'X']
 ['O' 'O' 'O' 'X' 'X' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'X']
 ['O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'X' 'O' 'O' 'O' 'O' 'O' 'O' 'X' 'O' 'O' 'O' 'O' 'O' 'O']
 ['O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O' 'O'